# GNN con Demográficos + Biomarcadores CSF (Solo datos reales)
Este notebook es una **versión mejorada** que usa únicamente visitas con biomarcadores **reales**.

**Diferencia clave con DemoAndBiomarkers.ipynb:**
- ❌ **Versión anterior**: Imputaba biomarcadores faltantes (~71% imputados)
- ✅ **Esta versión**: Solo usa visitas con biomarcadores medidos (~29% de datos, pero reales)

**Features:**
- Demographics: AGE_AT_VISIT, PTEDUCAT, PTGENDER, PTMARRY
- Biomarkers: ABETA42, TAU, PTAU
- Ratios: TAU/ABETA42, PTAU/ABETA42, PTAU/TAU

**Objetivo**: Predecir años hasta inicio de Alzheimer con datos de alta calidad

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

torch.manual_seed(42)
np.random.seed(42)

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
csv_path = "./data/adni/demographics/PTDEMOG.csv"
df = pd.read_csv(csv_path)
print(f"Demographics loaded: {df.shape}")

Demographics loaded: (6210, 84)


In [3]:
biomarker_path = "./data/adni/demographics/UPENNBIOMK_ROCHE_ELECSYS_11Oct2025.csv"
df_bio = pd.read_csv(biomarker_path)

print(f"Biomarkers loaded: {df_bio.shape}")
print(f"\nMissing values:")
print(df_bio[['ABETA42', 'TAU', 'PTAU']].isnull().sum())

bio_cols_to_use = ['RID', 'VISCODE2', 'EXAMDATE', 'ABETA42', 'TAU', 'PTAU']
df_bio = df_bio[bio_cols_to_use].copy()

for col in ['ABETA42', 'TAU', 'PTAU']:
    df_bio[col] = pd.to_numeric(df_bio[col], errors='coerce')

df_bio = df_bio.dropna(subset=['ABETA42', 'TAU', 'PTAU'])
print(f"\n✅ After filtering complete biomarkers: {df_bio.shape[0]} visits with real data")

Biomarkers loaded: (3174, 13)

Missing values:
ABETA42     7
TAU        15
PTAU       27
dtype: int64

✅ After filtering complete biomarkers: 3143 visits with real data


In [4]:
def norm_codes_to_labels(s: pd.Series, mapping: dict) -> pd.Series:
    out = s.astype(str).str.strip().str.replace(r"\.0$", "", regex=True)
    out = out.map(mapping)
    return out

gender_map = {"1":"male","2":"female","male":"male","female":"female","m":"male","f":"female"}
marry_map  = {"1":"married","2":"widowed","3":"divorced","4":"never_married","6":"domestic_partnership"}

def to_year(s):
    s = pd.to_numeric(s, errors="coerce")
    s = s.where((s >= 1900) & (s <= 2100))
    return s

In [5]:

onset_cols = [c for c in ["PTCOGBEG","PTADBEG","PTADDX"] if c in df.columns]
for c in onset_cols:
    df[c] = to_year(df[c])

def row_min_nonnull(row):
    vals = [row[c] for c in onset_cols if pd.notna(row[c])]
    return min(vals) if vals else np.nan

df["YEAR_ONSET"] = df.apply(row_min_nonnull, axis=1) if onset_cols else np.nan
df["YEAR_ONSET"] = to_year(df["YEAR_ONSET"])

for c in ["PTDOBYY","PTEDUCAT"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

if "PTGENDER" in df.columns:
    df["PTGENDER"] = norm_codes_to_labels(df["PTGENDER"], gender_map)
if "PTMARRY" in df.columns:
    df["PTMARRY"]  = norm_codes_to_labels(df["PTMARRY"], marry_map)

print(f"Demographics processed: {df.shape}")

Demographics processed: (6210, 85)


In [6]:

date_col = "EXAMDATE" if "EXAMDATE" in df.columns else ("VISDATE" if "VISDATE" in df.columns else None)
if not date_col:
    raise ValueError("No date column found")

df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
df_bio['EXAMDATE'] = pd.to_datetime(df_bio['EXAMDATE'], errors='coerce')

viscode_col = 'VISCODE2' if 'VISCODE2' in df.columns else ('VISCODE' if 'VISCODE' in df.columns else None)

if viscode_col:
    print(f"Merging using RID + {viscode_col}")
    
    df['VISCODE_NORMALIZED'] = df[viscode_col].astype(str).str.strip().replace({'sc': 'bl', 'f': 'bl', 'nan': ''})
    df_bio['VISCODE_NORMALIZED'] = df_bio['VISCODE2'].astype(str).str.strip()
    
    df_merged = df.merge(
        df_bio[['RID', 'VISCODE_NORMALIZED', 'ABETA42', 'TAU', 'PTAU']], 
        on=['RID', 'VISCODE_NORMALIZED'],
        how='inner',  # INNER JOIN: solo visitas con biomarcadores
        suffixes=('', '_bio')
    )
    
    df_merged = df_merged.drop(columns=['VISCODE_NORMALIZED'])
else:
    print("Using date-based merge")
    df_merged = pd.merge_asof(
        df.sort_values([date_col]),
        df_bio[['RID', 'EXAMDATE', 'ABETA42', 'TAU', 'PTAU']].sort_values('EXAMDATE'),
        left_on=date_col,
        right_on='EXAMDATE',
        by='RID',
        direction='nearest',
        tolerance=pd.Timedelta('30 days')  # Match estricto: máximo 30 días
    )
    df_merged = df_merged.dropna(subset=['ABETA42', 'TAU', 'PTAU'])

df = df_merged

print(f"\n✅ After merge (INNER JOIN - only real biomarkers): {df.shape}")
print(f"Rows with complete biomarkers: {df[['ABETA42', 'TAU', 'PTAU']].notna().all(axis=1).sum()}")
print(f"Unique patients with biomarkers: {df['RID'].nunique()}")

Merging using RID + VISCODE2

✅ After merge (INNER JOIN - only real biomarkers): (1780, 88)
Rows with complete biomarkers: 1780
Unique patients with biomarkers: 1622


In [7]:

df['TAU_ABETA42_RATIO'] = df['TAU'] / (df['ABETA42'] + 1e-6)
df['PTAU_ABETA42_RATIO'] = df['PTAU'] / (df['ABETA42'] + 1e-6)
df['PTAU_TAU_RATIO'] = df['PTAU'] / (df['TAU'] + 1e-6)

print("Feature engineering completed.")
print(f"Created ratios: TAU_ABETA42_RATIO, PTAU_ABETA42_RATIO, PTAU_TAU_RATIO")

Feature engineering completed.
Created ratios: TAU_ABETA42_RATIO, PTAU_ABETA42_RATIO, PTAU_TAU_RATIO


In [8]:

df["EXAM_YEAR"] = to_year(df[date_col].dt.year)

df["AGE_AT_VISIT"] = np.where(
    df["EXAM_YEAR"].notna() & df["PTDOBYY"].notna(),
    df["EXAM_YEAR"] - df["PTDOBYY"],
    np.nan
)

df["YEARS_TO_ONSET"] = np.where(
    df["YEAR_ONSET"].notna() & df["EXAM_YEAR"].notna(),
    df["YEAR_ONSET"] - df["EXAM_YEAR"],
    np.nan
)

df.loc[(df["YEARS_TO_ONSET"] < 0) & df["YEAR_ONSET"].notna(), "YEARS_TO_ONSET"] = np.nan
df.loc[df["YEARS_TO_ONSET"] > 50, "YEARS_TO_ONSET"] = np.nan

df["HAS_LABEL"] = df["YEARS_TO_ONSET"].notna()

print(f"\nVisits prepared: {len(df)}")
print(f"Labeled visits: {df['HAS_LABEL'].sum()}")
print(f"Unique patients: {df['RID'].nunique()}")


Visits prepared: 1780
Labeled visits: 23
Unique patients: 1622


In [9]:

biomarker_cols = ['ABETA42', 'TAU', 'PTAU', 'TAU_ABETA42_RATIO', 'PTAU_ABETA42_RATIO', 'PTAU_TAU_RATIO']
num_cols = [c for c in ["AGE_AT_VISIT", "PTEDUCAT"] + biomarker_cols if c in df.columns]
cat_cols = [c for c in ["PTGENDER", "PTMARRY"] if c in df.columns]

for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce").fillna(df[c].median())

parts = []

if num_cols:
    scaler = StandardScaler()
    X_num = pd.DataFrame(
        scaler.fit_transform(df[num_cols]),
        columns=num_cols,
        index=df.index
    )
    parts.append(X_num)
    print(f"Numeric features ({len(num_cols)}): {num_cols}")

if cat_cols:
    X_cat = pd.get_dummies(df[cat_cols], prefix=cat_cols, drop_first=False, dtype=float)
    parts.append(X_cat)
    print(f"Categorical features ({len(cat_cols)}): {cat_cols} -> {X_cat.shape[1]} columns")

X = pd.concat(parts, axis=1).astype(float)
X_clean = X.replace([np.inf, -np.inf], np.nan).fillna(0.0)

print(f"\n✅ Feature matrix (REAL biomarkers only): {X_clean.shape}")
print(f"Total features: {X_clean.shape[1]}")

Numeric features (8): ['AGE_AT_VISIT', 'PTEDUCAT', 'ABETA42', 'TAU', 'PTAU', 'TAU_ABETA42_RATIO', 'PTAU_ABETA42_RATIO', 'PTAU_TAU_RATIO']
Categorical features (2): ['PTGENDER', 'PTMARRY'] -> 6 columns

✅ Feature matrix (REAL biomarkers only): (1780, 14)
Total features: 14


In [10]:

n_samples = X_clean.shape[0]
k = min(8, max(1, n_samples - 1))
n_neighbors = min(n_samples, k + 1)

if n_samples >= 2:
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, metric="euclidean")
    nbrs.fit(X_clean.values)
    _, idx = nbrs.kneighbors(X_clean.values)
    src_knn, dst_knn = [], []
    for i in range(idx.shape[0]):
        for j in idx[i, 1:]:
            src_knn.append(i)
            dst_knn.append(j)
    edge_knn = torch.tensor([src_knn, dst_knn], dtype=torch.long)
else:
    edge_knn = torch.empty((2, 0), dtype=torch.long)

tmp = df.reset_index()[["index", "RID", date_col]].dropna(subset=[date_col]).sort_values(["RID", date_col])
src_tmp, dst_tmp = [], []
for rid, g in tmp.groupby("RID"):
    ids = g["index"].tolist()
    for a, b in zip(ids[:-1], ids[1:]):
        src_tmp.append(a)
        dst_tmp.append(b)
edge_tmp = torch.tensor([src_tmp, dst_tmp], dtype=torch.long) if src_tmp else torch.empty((2,0), dtype=torch.long)

def undirected(e):
    return torch.cat([e, e.flip(0)], dim=1) if e.numel() else e

edges = []
if edge_knn.numel(): edges.append(undirected(edge_knn))
if edge_tmp.numel(): edges.append(undirected(edge_tmp))
edge_index = torch.cat(edges, dim=1) if edges else torch.empty((2,0), dtype=torch.long)
if edge_index.numel():
    edge_index = torch.unique(edge_index, dim=1)

print(f"\nGraph: {len(df)} nodes, {edge_index.size(1)} edges")


Graph: 1780 nodes, 19808 edges


In [11]:

df["USE_FOR_LABEL"] = False
if df["HAS_LABEL"].any():
    idx_last_pre = df.loc[df["HAS_LABEL"]].groupby("RID")[date_col].idxmax()
    df.loc[idx_last_pre, "USE_FOR_LABEL"] = True

rids_with_label = df.loc[df["USE_FOR_LABEL"], "RID"].dropna().unique()
rng = np.random.default_rng(42)
rng.shuffle(rids_with_label)
n_lab_rids = len(rids_with_label)

tr_n = max(1, int(0.7 * n_lab_rids))
va_n = max(0, int(0.15 * n_lab_rids))
if tr_n + va_n > max(0, n_lab_rids - 1):
    va_n = max(0, n_lab_rids - 1 - tr_n)

train_rids = set(rids_with_label[:tr_n])
val_rids   = set(rids_with_label[tr_n:tr_n+va_n])
test_rids  = set(rids_with_label[tr_n+va_n:])

node_split = np.full(len(df), "train", dtype=object)
node_rids = df["RID"].to_numpy()
node_split[np.isin(node_rids, list(val_rids))]  = "val"
node_split[np.isin(node_rids, list(test_rids))] = "test"

use_for_label = df["USE_FOR_LABEL"].to_numpy()
train_mask_np = (node_split == "train") & use_for_label
val_mask_np   = (node_split == "val")   & use_for_label
test_mask_np  = (node_split == "test")  & use_for_label

split_map = {"train":0, "val":1, "test":2}
split_idx = np.vectorize(split_map.get)(node_split)
src_np = edge_index[0].cpu().numpy()
dst_np = edge_index[1].cpu().numpy()
keep_edges = split_idx[src_np] == split_idx[dst_np]
edge_index = edge_index[:, torch.tensor(keep_edges)]

print(f"\nSplits: train={len(train_rids)}, val={len(val_rids)}, test={len(test_rids)} RIDs")
print(f"Labels: train={train_mask_np.sum()}, val={val_mask_np.sum()}, test={test_mask_np.sum()}")


Splits: train=16, val=3, test=4 RIDs
Labels: train=16, val=3, test=4


In [12]:

y_full = df["YEARS_TO_ONSET"].astype(float)
y_mu  = float(y_full[train_mask_np].mean()) if train_mask_np.any() else 0.0
y_std = float(y_full[train_mask_np].std(ddof=0)) if train_mask_np.any() else 1.0
if not np.isfinite(y_std) or y_std == 0.0:
    y_std = 1.0

y_scaled = (y_full - y_mu) / y_std
y_t = torch.tensor(y_scaled.fillna(0).values, dtype=torch.float32)

x = torch.tensor(X_clean.values, dtype=torch.float32)
train_mask = torch.tensor(train_mask_np, dtype=torch.bool)
val_mask   = torch.tensor(val_mask_np,   dtype=torch.bool)
test_mask  = torch.tensor(test_mask_np,  dtype=torch.bool)

data = Data(x=x, edge_index=edge_index, y=y_t,
            train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)

print(f"\nData: x{data.x.shape}, edges{data.edge_index.shape}")
print(f"Target scaling: mean={y_mu:.3f}, std={y_std:.3f}")

Y_MEAN_TRAIN, Y_STD_TRAIN = y_mu, y_std


Data: xtorch.Size([1780, 14]), edgestorch.Size([2, 19642])
Target scaling: mean=0.000, std=1.000


In [13]:

class GNNRegressor(nn.Module):
    def __init__(self, in_ch, hid=64, dropout=0.3):
        super().__init__()
        self.c1 = GCNConv(in_ch, hid)
        self.c2 = GCNConv(hid, 1)
        self.dropout = dropout
        
    def forward(self, x, edge_index):
        x = F.relu(self.c1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.c2(x, edge_index)
        return x.squeeze(-1)

device = torch.device("cpu")
model = GNNRegressor(in_ch=data.num_node_features, hid=64, dropout=0.3).to(device)
data = data.to(device)

print(f"\nModel: {data.num_node_features} features -> 64 hidden -> 1 output")
print(f"Parameters: {sum(p.numel() for p in model.parameters())}")


Model: 14 features -> 64 hidden -> 1 output
Parameters: 1025


In [14]:

opt = torch.optim.AdamW(model.parameters(), lr=1e-2, weight_decay=1e-4)
loss_fn = nn.MSELoss()

def eval_metrics(split="val"):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        mask = data.train_mask if split=="train" else (data.val_mask if split=="val" else data.test_mask)
        if not mask.any():
            return float("nan"), float("nan")
        mae_scaled = torch.mean(torch.abs(out[mask] - data.y[mask])).item()
        rmse_scaled = torch.sqrt(loss_fn(out[mask], data.y[mask])).item()
        return mae_scaled * Y_STD_TRAIN, rmse_scaled * Y_STD_TRAIN

print("\nTraining...\n")

for ep in range(1, 101):
    model.train()
    opt.zero_grad()
    out = model(data.x, data.edge_index)
    
    if data.train_mask.any():
        loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
        if torch.isnan(loss) or torch.isinf(loss):
            raise RuntimeError("Loss NaN/Inf")
        loss.backward()
        opt.step()
        loss_val = loss.detach().item()
    else:
        loss_val = float("nan")
    
    if ep % 10 == 0:
        tr_mae, tr_rmse = eval_metrics("train")
        va_mae, va_rmse = eval_metrics("val")
        te_mae, te_rmse = eval_metrics("test")
        print(
            f"ep {ep:03d} | loss {loss_val:.4f} "
            f"| TR MAE {tr_mae:.3f}y RMSE {tr_rmse:.3f}y "
            f"| VAL MAE {va_mae:.3f}y RMSE {va_rmse:.3f}y "
            f"| TEST MAE {te_mae:.3f}y RMSE {te_rmse:.3f}y"
        )

print("\nTraining completed!")


Training...

ep 010 | loss 0.0137 | TR MAE 0.070y RMSE 0.080y | VAL MAE 0.110y RMSE 0.183y | TEST MAE 0.092y RMSE 0.106y
ep 020 | loss 0.0092 | TR MAE 0.065y RMSE 0.088y | VAL MAE 0.149y RMSE 0.202y | TEST MAE 0.088y RMSE 0.110y
ep 030 | loss 0.0205 | TR MAE 0.106y RMSE 0.140y | VAL MAE 0.226y RMSE 0.316y | TEST MAE 0.169y RMSE 0.199y
ep 040 | loss 0.0140 | TR MAE 0.080y RMSE 0.088y | VAL MAE 0.143y RMSE 0.210y | TEST MAE 0.134y RMSE 0.156y
ep 050 | loss 0.0115 | TR MAE 0.067y RMSE 0.094y | VAL MAE 0.063y RMSE 0.066y | TEST MAE 0.107y RMSE 0.114y
ep 060 | loss 0.0109 | TR MAE 0.044y RMSE 0.054y | VAL MAE 0.094y RMSE 0.126y | TEST MAE 0.039y RMSE 0.041y
ep 070 | loss 0.0014 | TR MAE 0.029y RMSE 0.035y | VAL MAE 0.111y RMSE 0.173y | TEST MAE 0.076y RMSE 0.090y
ep 080 | loss 0.0244 | TR MAE 0.050y RMSE 0.067y | VAL MAE 0.047y RMSE 0.079y | TEST MAE 0.063y RMSE 0.099y
ep 090 | loss 0.0108 | TR MAE 0.033y RMSE 0.043y | VAL MAE 0.099y RMSE 0.146y | TEST MAE 0.076y RMSE 0.090y
ep 100 | loss 

In [15]:

model.eval()
with torch.no_grad():
    final_out = model(data.x, data.edge_index)

tr_mae, tr_rmse = eval_metrics("train")
va_mae, va_rmse = eval_metrics("val")
te_mae, te_rmse = eval_metrics("test")

print("\n" + "="*70)
print("FINAL RESULTS (Real Biomarkers Only)")
print("="*70)
print(f"Dataset: {len(df)} visits from {df['RID'].nunique()} patients")
print(f"Features: {data.num_node_features} (demographics + real CSF biomarkers)")
print("\nPerformance:")
print(f"  TRAIN | MAE: {tr_mae:.3f} years | RMSE: {tr_rmse:.3f} years")
print(f"  VAL   | MAE: {va_mae:.3f} years | RMSE: {va_rmse:.3f} years")
print(f"  TEST  | MAE: {te_mae:.3f} years | RMSE: {te_rmse:.3f} years")
print("="*70)


FINAL RESULTS (Real Biomarkers Only)
Dataset: 1780 visits from 1622 patients
Features: 14 (demographics + real CSF biomarkers)

Performance:
  TRAIN | MAE: 0.039 years | RMSE: 0.062 years
  VAL   | MAE: 0.103 years | RMSE: 0.171 years
  TEST  | MAE: 0.088 years | RMSE: 0.113 years


In [16]:

import json

results = {
    'model': 'Demographics + CSF Biomarkers (Real Only)',
    'features': data.num_node_features,
    'train_mae': tr_mae,
    'train_rmse': tr_rmse,
    'val_mae': va_mae,
    'val_rmse': va_rmse,
    'test_mae': te_mae,
    'test_rmse': te_rmse,
    'n_visits': len(df),
    'n_patients': int(df['RID'].nunique())
}

with open('real_biomarkers_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\nResults saved to: real_biomarkers_results.json")
print("\nCompare with baseline using CompareModels.ipynb!")


Results saved to: real_biomarkers_results.json

Compare with baseline using CompareModels.ipynb!
